In [12]:
%reset -f

In [50]:
import pandas as pd

In [52]:
outputDir = '../../nlp/data/nlp/output/'
outFile = 'runNER_Output.tsv'
inputDir = '../../nlp/data/input/'
inFile = 'biosamples-expanded.tsv'
#cols = ['BiosampleId', 'Biolink', 'BeginTerm', 'EndTerm', 'TokenizedTerm', 'PreferredTerm', 'CURIE', 'NaN1', 'S-Type', 'NaN2', 'UMLS_CUI']
runnerOutput = pd.read_csv(outputDir+outFile, sep='\t' )
runnerOutput = runnerOutput.drop(['START POSITION', 'END POSITION', 'ZONE','SENTENCE ID', 'ORIGIN', 'UMLS CUI', 'SENTENCE'], axis = 1)
runnerOutput = runnerOutput.drop_duplicates()
runnerOutput.head()


,DOCUMENT ID,TYPE,MATCHED TERM,PREFERRED FORM,ENTITY ID
0,gold:Gb0101224,biolink:OntologyClass,Coal,coal,ENVO:02000091
1,gold:Gb0101224,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169
4,gold:Gb0101224,biolink:ChemicalSubstance,water,water,CHEBI:15377
5,gold:Gb0101224,biolink:OntologyClass,water,liquid water,ENVO:00002006_SYNONYM
6,gold:Gb0101225,biolink:OntologyClass,Coal,coal,ENVO:02000091


In [53]:
goldInput = pd.read_csv(inputDir+inFile, sep='\t', low_memory=False, usecols=['id', 'env_broad_scale', 'env_local_scale', 'env_medium'] )
goldInput.head()

,id,env_broad_scale,env_local_scale,env_medium
0,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792
1,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792
2,gold:Gb0101226,ENVO:00002030,ENVO:00002169,ENVO:00005792
3,gold:Gb0101227,ENVO:00002030,ENVO:00002169,ENVO:00005792
4,gold:Gb0101253,ENVO:00000446,ENVO:00002185,ENVO:00002985


In [54]:
inputOutputJoin = goldInput.merge(runnerOutput, left_on='id', right_on='DOCUMENT ID', how='left' )

In [55]:
inputOutputJoin = inputOutputJoin.drop(['DOCUMENT ID'], axis=1)
#inputOutputJoin = inputOutputJoin.drop_duplicates()
inputOutputJoin['CURIE'] = inputOutputJoin['ENTITY ID'].str.strip('_SYNONYM')
inputOutputJoin = inputOutputJoin.drop(['ENTITY ID'], axis=1)
inputOutputJoin.head()

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE
0,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091
1,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169
2,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:ChemicalSubstance,water,water,CHEBI:15377
3,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,water,liquid water,ENVO:00002006
4,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091


In [56]:
inputOutputJoin.to_csv(outputDir+'inputOutputJoined.tsv', sep='\t', index=None)

In [57]:
filteredMatch = inputOutputJoin.query('CURIE == env_broad_scale' and \
                                      'CURIE == env_local_scale' and \
                                      'CURIE == env_medium')

#filteredMatch = filteredMatch.drop_duplicates()

In [58]:
filteredMatch.head()

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE
17,gold:Gb0101253,ENVO:00000446,ENVO:00002185,ENVO:00002985,biolink:OntologyClass,oil,oil,ENVO:00002985
24,gold:Gb0101254,ENVO:00000446,ENVO:00002185,ENVO:00002985,biolink:OntologyClass,oil,oil,ENVO:00002985
1454,gold:Gb0102398,ENVO:00000447,ENVO:00000208,ENVO:03000033,biolink:OntologyClass,marine sediment,marine sediment,ENVO:03000033
1460,gold:Gb0102399,ENVO:00000447,ENVO:00000208,ENVO:03000033,biolink:OntologyClass,marine sediment,marine sediment,ENVO:03000033
1466,gold:Gb0102400,ENVO:00000447,ENVO:00000208,ENVO:03000033,biolink:OntologyClass,marine sediment,marine sediment,ENVO:03000033


In [59]:
filteredMatch.to_csv(outputDir+'filteredMatch.tsv', sep='\t', index=None)

In [60]:
len(inputOutputJoin)

208131

In [61]:
# env_{local,medium,broad}_matched
inputOutputJoin['env_broad_matched'] = inputOutputJoin['CURIE'] == inputOutputJoin['env_broad_scale']
inputOutputJoin['env_local_matched'] = inputOutputJoin['CURIE'] == inputOutputJoin['env_local_scale']
inputOutputJoin['env_medium_matched'] = inputOutputJoin['CURIE'] == inputOutputJoin['env_medium']
inputOutputJoin['false_positives'] = (inputOutputJoin.iloc[:,-3:]==False).all(True)

inputOutputJoin.head()

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,false_positives
0,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091,False,False,False,True
1,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169,False,True,False,False
2,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:ChemicalSubstance,water,water,CHEBI:15377,False,False,False,True
3,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,water,liquid water,ENVO:00002006,False,False,False,True
4,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091,False,False,False,True


In [62]:
grpList = ['id', 'env_broad_scale', 'env_local_scale', 'env_medium', 'TYPE', 'MATCHED TERM', 'PREFERRED FORM', 'CURIE']

collapsedIO  = inputOutputJoin.groupby(grpList).any()


In [63]:
collapsedIO.head()

env_broad_matched  \
id             env_broad_scale env_local_scale env_medium    TYPE                  MATCHED TERM PREFERRED FORM CURIE                              
gold:Gb0050971 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   
                                                                                   ecosystem    ecosystem      ENVO:01001110              False   
gold:Gb0050972 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   
                                                                                   ecosystem    ecosystem      ENVO:01001110              False   
gold:Gb0050973 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   

                                                                                                                              env_local_matched  \
id             env_broad_scale env_local_scale env_medium    TYPE                  MATCHED TERM PREFERRED FORM CURIE                              
gold:Gb0050971 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   
                                                                                   ecosystem    ecosystem      ENVO:01001110              False   
gold:Gb0050972 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   
                                                                                   ecosystem    ecosystem      ENVO:01001110              False   
gold:Gb0050973 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   

                                                                                                                              env_medium_matched  \
id             env_broad_scale env_local_scale env_medium    TYPE                  MATCHED TERM PREFERRED FORM CURIE                               
gold:Gb0050971 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998                True   
                                                                                   ecosystem    ecosystem      ENVO:01001110               False   
gold:Gb0050972 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998                True   
                                                                                   ecosystem    ecosystem      ENVO:01001110               False   
gold:Gb0050973 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998                True   

                                                                                                                              false_positives  
id             env_broad_scale env_local_scale env_medium    TYPE                  MATCHED TERM PREFERRED FORM CURIE                           
gold:Gb0050971 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998            False  
                                                                                   ecosystem    ecosystem      ENVO:01001110             True  
gold:Gb0050972 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998            False  
                                                                                   ecosystem    ecosystem      ENVO:01001110             True  
gold:Gb0050973 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998            False

In [64]:
collapsedIO.to_csv(outputDir+'inputOutputComparison.tsv', sep='\t')

In [65]:
len(collapsedIO)

169239

In [66]:
newDf = pd.read_csv(outputDir+'inputOutputComparison.tsv', sep='\t', low_memory=False)
newDf.head()

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,false_positives
0,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False
1,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,ecosystem,ecosystem,ENVO:01001110,False,False,False,True
2,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False
3,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,ecosystem,ecosystem,ENVO:01001110,False,False,False,True
4,gold:Gb0050973,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False


In [67]:
consolidatedDF = newDf.groupby(['id'])\
                        .agg({
                                'env_broad_scale': lambda a: ','.join(a),
                                'env_local_scale': lambda b: ','.join(b),
                                'env_medium': lambda c: ','.join(c),
                                'TYPE': lambda d: ','.join(d),
                                'MATCHED TERM': lambda e: ','.join(e),
                                'PREFERRED FORM': lambda f: ','.join(f),
                                'CURIE': lambda g: ','.join(g),
                                'env_broad_matched': lambda h: any(h == True),
                                'env_local_matched': lambda i: any(i == True),
                                'env_medium_matched': lambda j: any(j == True),
                                'false_positives': lambda k: any(k == False)
                            })
consolidatedDF.head()

,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,false_positives
id,,,,,,,,,,,
gold:Gb0050971,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True
gold:Gb0050972,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True
gold:Gb0050973,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True
gold:Gb0050974,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True
gold:Gb0050975,"ENVO:00002030,ENVO:00002030,ENVO:00002030,ENVO...","ENVO:00001997,ENVO:00001997,ENVO:00001997,ENVO...","ENVO:00002034,ENVO:00002034,ENVO:00002034,ENVO...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Acid Mine Drainage,Biofilm,Mine Drainage,...","acid,acid mine drainage,biofilm,mine drainage,...","CHEBI:37527,ENVO:00001997,ENVO:00002034,ENVO:0...",False,True,True,True


In [68]:
consolidatedDF.to_csv(outputDir+'AllGroupedById.tsv', sep='\t')

In [69]:
newDfSelect = newDf[['id','MATCHED TERM','TYPE', 'PREFERRED FORM', 'CURIE', 'false_positives']]
falsePosSubset = newDfSelect[newDfSelect['false_positives'] == True]
falsePosSubset.head()

,id,MATCHED TERM,TYPE,PREFERRED FORM,CURIE,false_positives
1,gold:Gb0050971,ecosystem,biolink:OntologyClass,ecosystem,ENVO:01001110,True
3,gold:Gb0050972,ecosystem,biolink:OntologyClass,ecosystem,ENVO:01001110,True
5,gold:Gb0050973,ecosystem,biolink:OntologyClass,ecosystem,ENVO:01001110,True
7,gold:Gb0050974,ecosystem,biolink:OntologyClass,ecosystem,ENVO:01001110,True
8,gold:Gb0050975,Acid,biolink:ChemicalSubstance,acid,CHEBI:37527,True


In [70]:
len(falsePosSubset)

146315

In [71]:
consolidatedFalseCURIEs = falsePosSubset.groupby(['id'])\
                        .agg({
                                'CURIE': lambda x: ','.join(x),
                                'TYPE': lambda a: ','.join(a),
                                'MATCHED TERM': lambda y: ','.join(y),
                                'PREFERRED FORM': lambda z: ','.join(z)
                            })\
                        .rename(columns={'CURIE':'falsePositiveCURIEs'})
consolidatedFalseCURIEs.head(10)

,falsePositiveCURIEs,TYPE,MATCHED TERM,PREFERRED FORM
id,,,,
gold:Gb0050971,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
gold:Gb0050972,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
gold:Gb0050973,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
gold:Gb0050974,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
gold:Gb0050975,"CHEBI:37527,ENVO:00001996,ENVO:00000083,ENVO:0...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Mine Drainage,Mountain,Mountain,Mountain,...","acid,mine drainage,hill,mount,mountain,seamoun..."
gold:Gb0050977,ENVO:00000015,biolink:OntologyClass,Ocean,ocean
gold:Gb0050978,"ENVO:01001788,ENVO:00000015,ENVO:01001110","biolink:OntologyClass,biolink:OntologyClass,bi...","Marine ecosystem,Ocean,ecosystem","marine ecosystem,ocean,ecosystem"
gold:Gb0050979,"ENVO:00000165,ENVO:00002277,ENVO:00000291,ENVO...","biolink:OntologyClass,biolink:OntologyClass,bi...","Basin,Basin,Basin,Basin,Basin,Basin,Basin,Fall...","asphalt lake,back-arc basin,drainage basin,ket..."
gold:Gb0050980,"UBERON:0001474,UBERON:0002481,ENVO:00000165,EN...","biolink:AnatomicalEntity,biolink:AnatomicalEnt...","Bone,Bone,Basin,Basin,Basin,Basin,Basin,Basin,...","bone element,bone tissue,asphalt lake,back-arc..."


In [72]:
consolidatedFalseCURIEs.to_csv(outputDir+'falsePositivesGroupedById.tsv', sep='\t')

In [73]:
falsePositives = pd.read_csv(outputDir+'falsePositivesGroupedById.tsv', sep='\t', low_memory=False)
falsePositives['false_positives'] = True
falsePositives.head()

,id,falsePositiveCURIEs,TYPE,MATCHED TERM,PREFERRED FORM,false_positives
0,gold:Gb0050971,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,True
1,gold:Gb0050972,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,True
2,gold:Gb0050973,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,True
3,gold:Gb0050974,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,True
4,gold:Gb0050975,"CHEBI:37527,ENVO:00001996,ENVO:00000083,ENVO:0...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Mine Drainage,Mountain,Mountain,Mountain,...","acid,mine drainage,hill,mount,mountain,seamoun...",True


In [74]:
newDf = newDf.drop(['CURIE', 'TYPE', 'MATCHED TERM', 'PREFERRED FORM'], axis=1)
finalDF = newDf.merge(falsePositives, on=['id', 'false_positives'],how='left')

In [75]:
goldInput = pd.read_csv('../../nlp/data/nlp/input/nlpGOLDInput.tsv', sep='\t', low_memory=False, index_col=None)
goldInput.head()

,id,text
0,gold:Gb0101224,Lithgow State Coal Mine Calcium nutrients (ear...
1,gold:Gb0101225,Lithgow State Coal Mine Calcium nutrients Extr...
2,gold:Gb0101226,Lithgow State Coal Mine Calcium nutrients. Lit...
3,gold:Gb0101227,Lithgow State Coal Mine Calcium merged assembl...
4,gold:Gb0101253,Bass Strait oil field metagenomics - A7A-2. Ba...


In [76]:
finalDF.to_csv(outputDir+'finalOutput.tsv', sep='\t', index=None)

In [77]:
onlyFalsePosivitesDF = newDf[newDf['false_positives'] == True].merge(falsePositives, on='id',how='inner')
onlyFalsePosivitesDF = onlyFalsePosivitesDF.drop(['false_positives_x', 'false_positives_y' ], axis=1)
onlyFalsePosivitesDF = onlyFalsePosivitesDF.drop_duplicates()

onlyFalsePosivitesDF.head(10)

,id,env_broad_scale,env_local_scale,env_medium,env_broad_matched,env_local_matched,env_medium_matched,falsePositiveCURIEs,TYPE,MATCHED TERM,PREFERRED FORM
0,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
1,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
2,gold:Gb0050973,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
3,gold:Gb0050974,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
4,gold:Gb0050975,ENVO:00002030,ENVO:00001997,ENVO:00002034,False,False,False,"CHEBI:37527,ENVO:00001996,ENVO:00000083,ENVO:0...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Mine Drainage,Mountain,Mountain,Mountain,...","acid,mine drainage,hill,mount,mountain,seamoun..."
11,gold:Gb0050977,ENVO:00000447,ENVO:00001999,ENVO:00002149,False,False,False,ENVO:00000015,biolink:OntologyClass,Ocean,ocean
12,gold:Gb0050978,ENVO:00002030,ENVO:00000098,ENVO:00002010,False,False,False,"ENVO:01001788,ENVO:00000015,ENVO:01001110","biolink:OntologyClass,biolink:OntologyClass,bi...","Marine ecosystem,Ocean,ecosystem","marine ecosystem,ocean,ecosystem"
15,gold:Gb0050979,ENVO:00000447,ENVO:01000105,ENVO:00002164,False,False,False,"ENVO:00000165,ENVO:00002277,ENVO:00000291,ENVO...","biolink:OntologyClass,biolink:OntologyClass,bi...","Basin,Basin,Basin,Basin,Basin,Basin,Basin,Fall...","asphalt lake,back-arc basin,drainage basin,ket..."
26,gold:Gb0050980,ENVO:00000447,ENVO:01000105,ENVO:00002164,False,False,False,"UBERON:0001474,UBERON:0002481,ENVO:00000165,EN...","biolink:AnatomicalEntity,biolink:AnatomicalEnt...","Bone,Bone,Basin,Basin,Basin,Basin,Basin,Basin,...","bone element,bone tissue,asphalt lake,back-arc..."
39,gold:Gb0050981,ENVO:00000447,ENVO:02000049,ENVO:00002164,False,False,False,"UBERON:0001474,UBERON:0002481,ENVO:00000040,EN...","biolink:AnatomicalEntity,biolink:AnatomicalEnt...","Bone,Bone,Fall,Peninsula,West,Whale Fall,micro...","bone element,bone tissue,waterfall,peninsula,w..."


In [78]:
onlyFalsePosWithText = onlyFalsePosivitesDF.merge(goldInput, on='id', how='inner')
onlyFalsePosWithText = onlyFalsePosWithText.drop_duplicates()
onlyFalsePosWithText.head(10)

,id,env_broad_scale,env_local_scale,env_medium,env_broad_matched,env_local_matched,env_medium_matched,falsePositiveCURIEs,TYPE,MATCHED TERM,PREFERRED FORM,text
0,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,Soil ecosystem from different sites within th...
1,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,Soil ecosystem from different sites within th...
2,gold:Gb0050973,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,Soil ecosystem from different sites within th...
3,gold:Gb0050974,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,Soil ecosystem from different sites within th...
4,gold:Gb0050975,ENVO:00002030,ENVO:00001997,ENVO:00002034,False,False,False,"CHEBI:37527,ENVO:00001996,ENVO:00000083,ENVO:0...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Mine Drainage,Mountain,Mountain,Mountain,...","acid,mine drainage,hill,mount,mountain,seamoun...",Acid Mine Drainage (ARMAN) microbial communiti...
5,gold:Gb0050977,ENVO:00000447,ENVO:00001999,ENVO:00002149,False,False,False,ENVO:00000015,biolink:OntologyClass,Ocean,ocean,Marine microbial communities from the Indian O...
6,gold:Gb0050978,ENVO:00002030,ENVO:00000098,ENVO:00002010,False,False,False,"ENVO:01001788,ENVO:00000015,ENVO:01001110","biolink:OntologyClass,biolink:OntologyClass,bi...","Marine ecosystem,Ocean,ecosystem","marine ecosystem,ocean,ecosystem",Marine ecosystem from Global Ocean Sampling (G...
7,gold:Gb0050979,ENVO:00000447,ENVO:01000105,ENVO:00002164,False,False,False,"ENVO:00000165,ENVO:00002277,ENVO:00000291,ENVO...","biolink:OntologyClass,biolink:OntologyClass,bi...","Basin,Basin,Basin,Basin,Basin,Basin,Basin,Fall...","asphalt lake,back-arc basin,drainage basin,ket...","Fossil microbial communities from Whale Fall, ..."
8,gold:Gb0050980,ENVO:00000447,ENVO:01000105,ENVO:00002164,False,False,False,"UBERON:0001474,UBERON:0002481,ENVO:00000165,EN...","biolink:AnatomicalEntity,biolink:AnatomicalEnt...","Bone,Bone,Basin,Basin,Basin,Basin,Basin,Basin,...","bone element,bone tissue,asphalt lake,back-arc...","Fossil microbial communities from Whale Fall, ..."
9,gold:Gb0050981,ENVO:00000447,ENVO:02000049,ENVO:00002164,False,False,False,"UBERON:0001474,UBERON:0002481,ENVO:00000040,EN...","biolink:AnatomicalEntity,biolink:AnatomicalEnt...","Bone,Bone,Fall,Peninsula,West,Whale Fall,micro...","bone element,bone tissue,waterfall,peninsula,w...","Fossil microbial communities from Whale Fall, ..."


In [79]:
onlyFalsePosWithText.to_csv(outputDir+'onlyFalsePosWithText.tsv', sep='\t', index=None)

In [80]:
allDf = pd.read_csv(outputDir+'AllGroupedById.tsv', sep='\t', low_memory=False)
allDf.head()

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,false_positives
0,gold:Gb0050971,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True
1,gold:Gb0050972,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True
2,gold:Gb0050973,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True
3,gold:Gb0050974,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True
4,gold:Gb0050975,"ENVO:00002030,ENVO:00002030,ENVO:00002030,ENVO...","ENVO:00001997,ENVO:00001997,ENVO:00001997,ENVO...","ENVO:00002034,ENVO:00002034,ENVO:00002034,ENVO...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Acid Mine Drainage,Biofilm,Mine Drainage,...","acid,acid mine drainage,biofilm,mine drainage,...","CHEBI:37527,ENVO:00001997,ENVO:00002034,ENVO:0...",False,True,True,True


In [81]:
allWithTextDF = allDf.merge(goldInput, on='id', how='inner')
allWithTextDF.head()

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,false_positives,text
0,gold:Gb0050971,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True,Soil ecosystem from different sites within th...
1,gold:Gb0050972,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True,Soil ecosystem from different sites within th...
2,gold:Gb0050973,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True,Soil ecosystem from different sites within th...
3,gold:Gb0050974,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110",False,False,True,True,Soil ecosystem from different sites within th...
4,gold:Gb0050975,"ENVO:00002030,ENVO:00002030,ENVO:00002030,ENVO...","ENVO:00001997,ENVO:00001997,ENVO:00001997,ENVO...","ENVO:00002034,ENVO:00002034,ENVO:00002034,ENVO...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Acid Mine Drainage,Biofilm,Mine Drainage,...","acid,acid mine drainage,biofilm,mine drainage,...","CHEBI:37527,ENVO:00001997,ENVO:00002034,ENVO:0...",False,True,True,True,Acid Mine Drainage (ARMAN) microbial communiti...


In [82]:
allWithTextDF.to_csv(outputDir+'AllWithText.tsv', sep='\t', index=None)